# Проверка на вхождение в список Росфинмониторинга

In [ ]:
from datetime import datetime
import pandas as pd
import xml.etree.ElementTree as ET

# Парсинг XML данных 
tree = ET.parse('data/16.12.2024-v 2.1.xml')
root = tree.getroot()

display(root)



АктуальныйПеречень
    Субъект
        ИдСубъекта {}
        ТипСубъекта {}
        История {}
        ФЛ {}
            Идентификатор {}
            Наименование {}
            ДатаВключения {}
            ФИО {}
            Фамилия {}
            Имя {}
            Отчество {}
            ФИОЛат {}
            ДатаРождения {}
            ГодРождения {}
            МестоРождения {}
            СписокДокументов {}
                Документ {}
                    Серия {} 6100
                    Номер {} 326012
                    ОрганВыдачи {} ВЫДАН МО УФМС РФ ПО РОСТОВСКОЙ ОБЛАСТИ В Г. ЗЕРНОГРАДЕ
                    ДатаВыдачи {} 2012-07-21
                    ТипДокумента {} 
                        Идентификатор {} 1631726
                        Наименование {} ПАСПОРТ РФ
                
            СписокГражданств {}
            Адрес {}
            
        СписокАдресов {}
        Примечание {} В РОЗЫСКЕ
        Террорист {} 1 или 0
        

In [ ]:
# Преобразование данных в список словарей 
data = [] 

for child in root:
    if child.tag == 'АктуальныйПеречень':
        for child2 in child:
            if child2.tag == 'Субъект':
                subject_id = None
                remark = None
                terr_marker = None
                fullname = None
                surname = None
                name = None
                secondname = None
                fullname_lat = None
                birth_date = None
                birth_year = None
                birth_place = None
                doc_series = None
                doc_number = None
                doc_issue_department = None
                doc_issue_date = None
                doc_type = None
                append_flag = False
                
                for child3 in child2: # Перебираем субъектов
                    append_flag = False
                    
                    subject_id_subj = child2.find('ИдСубъекта')
                    if not subject_id_subj is None: 
                        subject_id = subject_id_subj.text
                        append_flag = True
                    remark_subj = child2.find('Примечание')
                    if not remark_subj is None: 
                        remark = remark_subj.text
                        append_flag = True
                    terr_marker_subj = child2.find('Террорист')
                    if not terr_marker_subj is None: 
                        terr_marker = terr_marker_subj.text
                        append_flag = True

                    if child3.tag == 'ФЛ':
                        for child4 in child3:
                            if child4.tag == 'ФИО': 
                                fullname = child4.text
                                append_flag = True
                            if child4.tag == 'Фамилия': 
                                surname = child4.text
                                append_flag = True
                            if child4.tag == 'Имя': 
                                name = child4.text
                                append_flag = True
                            if child4.tag == 'Отчество': 
                                secondname = child4.text
                                append_flag = True
                            if child4.tag == 'ФИОЛат': 
                                fullname_lat = child4.text
                                append_flag = True
                            if child4.tag == 'ДатаРождения': 
                                birth_date = child4.text
                                append_flag = True
                            if child4.tag == 'МестоРождения': 
                                birth_place = child4.text
                                append_flag = True
                            if child4.tag == 'ГодРождения': 
                                birth_year = child4.text
                                append_flag = True
                                
                            if child4.tag == 'СписокДокументов':
                        
                                for child5 in child4: # Документ в Списке документов
                                
                                    for child6 in child5:
                                        
                                        if child6.tag == 'Серия': 
                                            doc_series = child6.text
                                            append_flag = True
                                        if child6.tag == 'Номер': 
                                            doc_number = child6.text
                                            append_flag = True
                                        if child6.tag == 'ОрганВыдачи': 
                                            doc_issue_department = child6.text
                                            append_flag = True
                                        if child6.tag == 'ДатаВыдачи': 
                                            doc_issue_date = child6.text
                                            append_flag = True
                                                                                
                                        if child6.tag == 'ТипДокумента':
                                            for child7 in child6:
                                                if child7.tag == 'Наименование': 
                                                    doc_type = child7.text
                                                    append_flag = True
                                
                                    if append_flag :
                                        data.append({
                                            'subject_id': subject_id,
                                            'fullname': fullname, 
                                            'surname': surname,                             
                                            'name': name, 
                                            'secondname': secondname,
                                            'fullname_lat': fullname_lat,
                                            'birth_date': birth_date,
                                            'birth_year': birth_year,
                                            'birth_place': birth_place,
                                            'doc_series': doc_series, 
                                            'doc_number': doc_number, 
                                            'doc_full_number': str(doc_series) + str(doc_number),
                                            'doc_issue_date': doc_issue_date,
                                            'doc_issue_department': doc_issue_department,
                                            'doc_type': doc_type,
                                            'remark': remark,
                                            'terr_marker': terr_marker
                                            })
                                        append_flag = False

                                   
display(data)                      


In [ ]:
# Создание DataFrame из списка словарей 
df = pd.DataFrame(data)

df.head(20)

In [ ]:
df.info()